# Foyer Tutorial 01: SMARTS and Overrides

This tutorial focuses on the basic usage of SMARTS to define chemical context and the use of overrides to set rule precedence.

As shown in the prior tutorial, atom-type usage (i.e., chemical context) is defined via SMARTS string contained in the `def` tag.  Below is the AtomTypes subsection of the XML file for a forcefield that describes alkanes using the OPLS all atom forcefield (as detailed in [Jorgensen, et al.](dx.doi.org/10.1021/ja9621760)). 

#### ! sed -n 2,18p OPLSaa_alkanes.xml

### Defining SMARTS
Focusing first on atom type `opls_140`, the SMARTS string, `def="H[C;X4]"`, states that this atom-type applies when:
- The element is hydrogen, i.e., `H`
- When that hydrogen is connected to a carbon that has 4 neighbors, i.e., `[C;X4]`

Similarly, for atom type `opls_138`, the SMARTS string, `def="[C;X4](H)(H)(H)H"`, states that this atom-type applies when:
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- 4 of those neighbors are hydrogens, i.e., `(H)(H)(H)H`

For atom type `opls_136`, the SMARTS string, `def="[C;X4](H)H"`, states that this atom-type applies when:
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- At least 2 of those neighbors are hydrogens, i.e., `(H)H`

For atom type `opls_135`, the SMARTS string, `def="[C;X4](H)(H)H"`, states that this atom-type applies when:
- The element is carbon, with 4 neighbors, i.e., `[C;X4]`
- At least 3 of those neighbors are hydrogens, i.e., `(H)(H)H`

### Atomtyping using SMARTS
Let us now consider using these rules to atom-type the carbon in methane (i.e., CH4).

opls_138 would obviously evaluate to `True`, as it is defined for a carbon, with 4 hydrogen neighbors. However, opls_135 and opls_136 would also evaulate to `True`.  In the case of opls_135, our definition only states that at least 3 of the neighbors are hydrogen, and have not made any specific claims about the identity of the 4th neighbor; similarly, for opls_136, we have only stated that 2 neighbors must be hydrogen and not specified the identity of the other 2 neighbors. 

### Overrides
We will discuss two ways to address this. One approach is to employ `overrides`.  `overrides` provide a means to dictate rule precedence (i.e., which rules are more specific than others).  In the forcefield field file above, opls_138 has defined: `overrides="opls_135,opls_136"`.  That is, if opls_138 evaluates to `True`, then it takes precedence over opls_135 and opls_136. 

Similarly, if opls_136 evaluates to `True`, it `overrides="opls_136"`, thus taking precedence. 

`overrides` are especially useful if the chemical context of two different atom-types are effectively the same.  E.g., the terminal methyl group in an alkane has the same first neighbor environment as the methyl group in toluene, however the parameters (and thus atom-type) are different. Thus `overrides` can be used to state that if the more specific toluene rule evaluates to `True` it should take precedences over the more general alkane rule (as shown below):

<img src="ch3-toluene.png" alt="Drawing" style="width: 700px;"/>

### Better SMARTS definitions
In many cases, `overrides` can be avoided by simply providing more specific SMARTS.  For example, the rules for opls_135, opls_136, and opls_138 above can be made more specific (in this case, stating the identify of the other neighbors besides carbon) and thus eliminate the need for `overrides`, as shown below. 

In [4]:
! sed -n 2,18p OPLSaa_alkanes2.xml

    <AtomTypes>
        <Type name="opls_135" def="[C;X4](H)(H)(H)C"
              class="CT" element="C" mass="12.01100" desc="alkane CH3"
              doi="10.1021/ja9621760"/>

        <Type name="opls_136" def="[C;X4](H)(H)(C)C"
              class="CT" element="C" mass="12.01100" desc="alkane CH2"
              doi="10.1021/ja9621760"/>

        <Type name="opls_138" def="[C;X4](H)(H)(H)H"
              class="CT" element="C" mass="12.01100" desc="alkane CH4"
              doi="10.1021/ja9621760"/>

        <Type name="opls_140" def="H[C;X4]"
              class="HC" element="H" mass="1.00800" desc="alkane H"
              doi="10.1021/ja9621760"/>
    </AtomTypes>


Note that SMARTS can be used to define more than just the immediate local pattern. Recall the definition for opls_140, `def='H[C;X4]'`. This difition states not only that the element is hydrogen and has a neighbor that is carbon, but that carbon has 4 bonds. If necessarily, the identify of those bonded neighbors could additionally be specified. 

Additionally, we note that since this forcefield only has a single hydrogen atom-type, the definition could actually be made less specific, i.e., `def='H'`, and still produce the correct output.  

Thus, it is important to keep in mind that there are multiple valid SMARTS definitions that can be provided for a given atom-type; the specificity of the SMARTS definition and whether or not overrides are necessary will depend on the chemical context of the parameters themselves, the number of atom-types included in a forcefield, the intended usage of the forcefield, and the personal preferences of the individual(s) defining the forcefield.   